In [108]:
#Import libraries

import pandas as pd
import os
import re

#Load data

file_path = '/Users/sherryarora/Desktop/netflix_titles.csv'
netflix_data = pd.read_csv(file_path)
print(netflix_data.head())

  show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water              NaN   
2      s3  TV Show              Ganglands  Julien Leclercq   
3      s4  TV Show  Jailbirds New Orleans              NaN   
4      s5  TV Show           Kota Factory              NaN   

                                                cast        country  \
0                                                NaN  United States   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa   
2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...            NaN   
3                                                NaN            NaN   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...          India   

           date_added  release_year rating   duration  \
0  September 25, 2021          2020  PG-13     90 min   
1  September 24, 2021          2021  TV-MA  2 Seasons   
2  September 24, 2021        

In [109]:
print(netflix_data.info())
print(netflix_data['rating'].unique())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB
None
['PG-13' 'TV-MA' 'PG' 'TV-14' 'TV-PG' 'TV-Y' 'TV-Y7' 'R' 'TV-G' 'G'
 'NC-17' '74 min' '84 min' '66 min' 'NR' nan 'TV-Y7-FV' 'UR']


In [111]:
# Convert date_added to date, duration cannot be put into 1 format since 1 season = ? mins , convert the rating which has min into 'Unknown',
# split countries, listed in, director and cast to separate rows to be able to viz based on the different countries and cast members
# Convert nulls to 'Unknown'
print(netflix_data.isnull().sum())


show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64


In [112]:
netflix_data['director'] = netflix_data['director'].fillna('Unknown')
netflix_data['cast'] = netflix_data['cast'].fillna('Unknown')
netflix_data['country'] = netflix_data['country'].fillna('Unknown')
netflix_data['rating'] = netflix_data['rating'].fillna('Unknown')
netflix_data['duration'] = netflix_data['duration'].fillna('Unknown')

# Count Unknown values in each column to check they match the nulls above
unknown_counts = {
    'director': (netflix_data['director'] == 'Unknown').sum(),
    'cast': (netflix_data['cast'] == 'Unknown').sum(),
    'country': (netflix_data['country'] == 'Unknown').sum(),
    'rating': (netflix_data['rating'] == 'Unknown').sum(),
    'duration': (netflix_data['duration'] == 'Unknown').sum()
}

# Print the counts to check
print(unknown_counts)


{'director': np.int64(2634), 'cast': np.int64(825), 'country': np.int64(831), 'rating': np.int64(4), 'duration': np.int64(3)}


In [113]:
"""
netflix_data['date_added'] = pd.to_datetime(netflix_data['date_added'], errors='coerce')
 nan_count = netflix_data['date_added'].isna().sum()
 print(nan_count) """
# Should be 10 nulls, not 98 post conversion. Above code no longer required

# Re do the date conversion to analyse and preserve the 88 dates (reload dataset first). 
"""

original_date_added = netflix_data['date_added'].copy()
netflix_data['date_added'] = pd.to_datetime(netflix_data['date_added'], errors='coerce')
invalid_dates = netflix_data[netflix_data['date_added'].isna()]

invalid_dates_with_original = invalid_dates[['show_id']].copy() 
invalid_dates_with_original['original_date_added'] = original_date_added[invalid_dates.index]  

print(invalid_dates_with_original) """

# Above code no longer required as we need to handle the date another way post above analysis

"\n\noriginal_date_added = netflix_data['date_added'].copy()\nnetflix_data['date_added'] = pd.to_datetime(netflix_data['date_added'], errors='coerce')\ninvalid_dates = netflix_data[netflix_data['date_added'].isna()]\n\ninvalid_dates_with_original = invalid_dates[['show_id']].copy() \ninvalid_dates_with_original['original_date_added'] = original_date_added[invalid_dates.index]  \n\nprint(invalid_dates_with_original) "

In [114]:
# Checked above printed values in csv, leading white space prior to these dates. Clean that up below (Reloaded dataset)
netflix_data['date_added'] = netflix_data['date_added'].str.strip()
netflix_data['date_added'] = pd.to_datetime(netflix_data['date_added'], errors='coerce', format='%B %d, %Y')



In [115]:
# Check for null values in the date_added column
invalid_dates = netflix_data[netflix_data['date_added'].isna()]
print(invalid_dates[['show_id', 'date_added']])   
# 10 nulls as expected 

     show_id date_added
6066   s6067        NaT
6174   s6175        NaT
6795   s6796        NaT
6806   s6807        NaT
6901   s6902        NaT
7196   s7197        NaT
7254   s7255        NaT
7406   s7407        NaT
7847   s7848        NaT
8182   s8183        NaT


In [116]:
# Put a anomaly date instead of nulls to format the column to one data type
netflix_data['date_added'] = netflix_data['date_added'].fillna(pd.to_datetime('1900-01-01'))

In [117]:
print(netflix_data.isnull().sum())
# No nulls as expected in any col


show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64


In [118]:
# Replace any rating values containing the str min with Unknown
netflix_data['rating'] = netflix_data['rating'].apply(lambda x: 'Unknown' if 'min' in str(x) else x)

unique_ratings = netflix_data['rating'].unique()
print("Unique ratings now:", unique_ratings)


Unique ratings now: ['PG-13' 'TV-MA' 'PG' 'TV-14' 'TV-PG' 'TV-Y' 'TV-Y7' 'R' 'TV-G' 'G'
 'NC-17' 'Unknown' 'NR' 'TV-Y7-FV' 'UR']


In [119]:
# To split cast, country and listen_in into separate rows to be able to visualise later and group by individual cast members or country or genre

# Function to clean names 
def clean_name(name):
    if isinstance(name, str):
        cleaned_name = re.sub(r'[^a-zA-ZáéíóúñÁÉÍÓÚÑüÜçÇ\s,]', '', name)
        return cleaned_name.strip()
    else:
        return ""  

# Clean the cast, country, listed_in, and director columns (before splitting and exploding)
netflix_data['cast'] = netflix_data['cast'].apply(lambda x: clean_name(str(x).replace("[", "").replace("]", "").replace("'", "").strip()))
netflix_data['country'] = netflix_data['country'].apply(lambda x: str(x).replace("[", "").replace("]", "").replace("'", "").strip())
netflix_data['listed_in'] = netflix_data['listed_in'].apply(lambda x: str(x).replace("[", "").replace("]", "").replace("'", "").strip())
netflix_data['director'] = netflix_data['director'].apply(lambda x: clean_name(str(x).replace("[", "").replace("]", "").replace("'", "").strip()))

# Split columns by commas into lists
netflix_data['cast'] = netflix_data['cast'].str.split(', ')
netflix_data['country'] = netflix_data['country'].str.split(', ')
netflix_data['listed_in'] = netflix_data['listed_in'].str.split(', ')
netflix_data['director'] = netflix_data['director'].str.split(', ')

# Exploding the columns to separate rows
netflix_data_exploded = netflix_data.explode('cast').explode('country').explode('listed_in').explode('director').reset_index(drop=True)

# Remove any remaining unwanted characters after exploding
netflix_data_exploded['cast'] = netflix_data_exploded['cast'].apply(lambda x: clean_name(str(x).strip()))
netflix_data_exploded['country'] = netflix_data_exploded['country'].apply(lambda x: str(x).strip())
netflix_data_exploded['listed_in'] = netflix_data_exploded['listed_in'].apply(lambda x: str(x).strip())
netflix_data_exploded['director'] = netflix_data_exploded['director'].apply(lambda x: clean_name(str(x).strip()))

# Validating data by checking random movies
movies_data = netflix_data_exploded[netflix_data_exploded['title'].isin(['Blood & Water', 'D.P.'])]
print(movies_data[['title', 'cast', 'country', 'listed_in', 'director']])



              title          cast       country               listed_in  \
1     Blood & Water    Ama Qamata  South Africa  International TV Shows   
2     Blood & Water    Ama Qamata  South Africa               TV Dramas   
3     Blood & Water    Ama Qamata  South Africa            TV Mysteries   
4     Blood & Water   Khosi Ngema  South Africa  International TV Shows   
5     Blood & Water   Khosi Ngema  South Africa               TV Dramas   
...             ...           ...           ...                     ...   
4748           D.P.  Kim Sungkyun   South Korea               TV Dramas   
4749           D.P.     Son Sukku                International TV Shows   
4750           D.P.     Son Sukku                             TV Dramas   
4751           D.P.     Son Sukku   South Korea  International TV Shows   
4752           D.P.     Son Sukku   South Korea               TV Dramas   

     director  
1     Unknown  
2     Unknown  
3     Unknown  
4     Unknown  
5     Unknown  
...

In [104]:
# Filter to find empty strings since DP above has empty string in countries
empty_values = netflix_data_exploded[(netflix_data_exploded['cast'] == '') | (netflix_data_exploded['country'] == '')]
print(empty_values[['title', 'cast', 'country', 'listed_in']])


                title                 cast country               listed_in
4737             D.P.           Jung Haein          International TV Shows
4738             D.P.           Jung Haein                       TV Dramas
4741             D.P.          Koo Kyohwan          International TV Shows
4742             D.P.          Koo Kyohwan                       TV Dramas
4745             D.P.         Kim Sungkyun          International TV Shows
4746             D.P.         Kim Sungkyun                       TV Dramas
4749             D.P.            Son Sukku          International TV Shows
4750             D.P.            Son Sukku                       TV Dramas
9056  Eyes of a Thief  Khaled Abol El Naga                          Dramas
9057  Eyes of a Thief  Khaled Abol El Naga              Independent Movies
9058  Eyes of a Thief  Khaled Abol El Naga            International Movies
9065  Eyes of a Thief          Souad Massi                          Dramas
9066  Eyes of a Thief    

In [105]:
# Drop rows where contains empty strings
netflix_data_cleaned = netflix_data_exploded[(netflix_data_exploded['cast'] != '') & (netflix_data_exploded['country'] != '')]
filtered_data = netflix_data_cleaned[netflix_data_cleaned['title'].isin(['Eyes of a Thief', 'D.P.'])]
print(filtered_data[['title', 'cast', 'country', 'listed_in']])



                title                 cast      country  \
4739             D.P.           Jung Haein  South Korea   
4740             D.P.           Jung Haein  South Korea   
4743             D.P.          Koo Kyohwan  South Korea   
4744             D.P.          Koo Kyohwan  South Korea   
4747             D.P.         Kim Sungkyun  South Korea   
4748             D.P.         Kim Sungkyun  South Korea   
4751             D.P.            Son Sukku  South Korea   
4752             D.P.            Son Sukku  South Korea   
9059  Eyes of a Thief  Khaled Abol El Naga       France   
9060  Eyes of a Thief  Khaled Abol El Naga       France   
9061  Eyes of a Thief  Khaled Abol El Naga       France   
9062  Eyes of a Thief  Khaled Abol El Naga      Algeria   
9063  Eyes of a Thief  Khaled Abol El Naga      Algeria   
9064  Eyes of a Thief  Khaled Abol El Naga      Algeria   
9068  Eyes of a Thief          Souad Massi       France   
9069  Eyes of a Thief          Souad Massi       France 

In [120]:
from nltk.corpus import stopwords
from nltk import download
download('stopwords')
stop_words = set(stopwords.words('english'))

# Function to clean description
def clean_description(description):
    description = description.lower()
    description = re.sub(r'[^a-z\s]', '', description)
    words = description.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

netflix_data_cleaned = netflix_data_cleaned.copy()

netflix_data_cleaned.loc[:, 'description_clean'] = netflix_data_cleaned['description'].apply(clean_description)

print(netflix_data_cleaned[['title', 'description_clean']])


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sherryarora/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                       title  \
0       Dick Johnson Is Dead   
1              Blood & Water   
2              Blood & Water   
3              Blood & Water   
4              Blood & Water   
...                      ...   
201986                Zubaan   
201987                Zubaan   
201988                Zubaan   
201989                Zubaan   
201990                Zubaan   

                                        description_clean  
0       father nears end life filmmaker kirsten johnso...  
1       crossing paths party cape town teen sets prove...  
2       crossing paths party cape town teen sets prove...  
3       crossing paths party cape town teen sets prove...  
4       crossing paths party cape town teen sets prove...  
...                                                   ...  
201986  scrappy poor boy worms way tycoons dysfunction...  
201987  scrappy poor boy worms way tycoons dysfunction...  
201988  scrappy poor boy worms way tycoons dysfunction...  
201989  scrappy

In [107]:
netflix_data_cleaned.to_csv('netflix_data_clean.csv', index=False)
